In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import random

In [12]:
data = pd.read_csv("dataset.csv")

In [13]:
def simple_backtest(returns, pnl, date, time, price, fee, volume, trading_rule, initial_balance=10000):
    signals = trading_rule(returns, pnl, date, time, price)
    positions = signals.diff().fillna(0)
    
    backtest_results = pd.DataFrame({
        'Date': date,
        'Time': time,
        'Price': price,
        'Signal': signals,
        'Position': positions
    })
    
    backtest_results['PnL'] = backtest_results['Position'] * backtest_results['Price'] - fee * abs(backtest_results['Position'].diff())
    backtest_results['CumulativePnL'] = backtest_results['PnL'].cumsum()
    backtest_results['Balance'] = initial_balance + backtest_results['CumulativePnL']
    
    return backtest_results

def moving_average_crossover(returns, pnl, date, time, price, short_window=50, long_window=200):
    short_ma = returns.rolling(window=short_window).mean()
    long_ma = returns.rolling(window=long_window).mean()
    signals = pd.Series(0, index=returns.index)
    signals[short_ma > long_ma] = 1
    signals[short_ma <= long_ma] = -1
    return signals

data = pd.read_csv('dataset.csv')
returns = data['return']
pnl = data['pnl']
date = pd.to_datetime(data['date'])
time = pd.to_timedelta(data['time'])
price = data['price']
fee = data['fee'].mean()
volume = data['volume'].mean()

backtest_results = simple_backtest(returns, pnl, date, time, price, fee, volume, moving_average_crossover, initial_balance=10000)
backtest_results

,Date,Time,Price,Signal,Position,PnL,CumulativePnL,Balance
0,2022-11-16,0 days 15:49:12,0.026925,0,0.0,NaN,NaN,NaN
1,2022-11-16,0 days 16:56:29,0.045741,0,0.0,0.0,0.000000e+00,1.000000e+04
2,2022-11-16,0 days 19:06:07,0.019886,0,0.0,0.0,0.000000e+00,1.000000e+04
3,2022-11-18,0 days 00:10:51,0.012323,0,0.0,0.0,0.000000e+00,1.000000e+04
4,2022-11-18,0 days 06:13:48,0.029858,0,0.0,0.0,0.000000e+00,1.000000e+04
...,...,...,...,...,...,...,...,...
26103,2023-02-08,0 days 09:10:47,-0.018810,-1,0.0,-0.0,-2.813564e+06,-2.803564e+06
26104,2023-02-12,0 days 16:35:16,0.025764,-1,0.0,0.0,-2.813564e+06,-2.803564e+06
26105,2023-02-14,0 days 23:56:51,0.022979,-1,0.0,0.0,-2.813564e+06,-2.803564e+06
26106,2023-03-04,0 days 18:06:25,0.020906,-1,0.0,0.0,-2.813564e+06,-2.803564e+06


In [14]:
def volume_copying_strategy_backtest(trader_df, trader, volume_to_copy, account_balance, random_volume=False):
    trader_df = trader_df[trader_df['trader'] == trader]
    copied_trades = pd.DataFrame(columns=['date', 'time', 'volume', 'pnl', 'pnl%', 'balance'])
    dates = []
    accountBalance = []
    if random_volume:
        volume_to_trade = trader_df[trader_df['trader'] == trader]['volume'].values.tolist()
        random.shuffle(volume_to_trade)
        for index, trade in trader_df.iterrows():
            volume = volume_to_trade.pop()
            trade_cost = trade['return'] * volume
            if trade_cost <= account_balance:
                initial = account_balance
                account_balance += trade_cost
                dates.append(trade['date'] + " " + trade['time'])
                accountBalance.append(account_balance)
                copied_trades.loc[len(copied_trades.index)] = [trade['date'], trade['time'], volume, trade_cost, trade_cost / initial, account_balance]
    else:
        volume_to_trade = min(trader_df[trader_df['trader'] == trader]['volume'].mean(), volume_to_copy)
        for index, trade in trader_df.iterrows():
            trade_cost = trade['return'] * volume_to_trade
            if trade_cost <= account_balance:
                initial = account_balance
                account_balance += trade_cost
                dates.append(trade['date'] + " " + trade['time'])
                accountBalance.append(account_balance)
                copied_trades.loc[len(copied_trades.index)] = [trade['date'], trade['time'], volume_to_trade, trade_cost, trade_cost / initial, account_balance]

    x_values = range(len(dates))
    plt.figure(figsize=(10, 6))
    plt.plot(x_values, accountBalance, marker='o', markersize=3, linestyle='-', color='b', label='Equity Curve')
    plt.xlabel('Date and Time')
    plt.ylabel('Account Balance')
    plt.title('Equity Curve')
    plt.xticks(x_values, dates, rotation=90)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    return copied_trades

In [15]:
volume_copying_strategy_backtest(data, '0xbbd2498a9e42af43062c1de268c8413601e1f8e4', 5000, 100000)

NameError: name 'initial_balance' is not defined

In [ ]:
volume_copying_strategy_backtest(data, '0xbbd2498a9e42af43062c1de268c8413601e1f8e4', 5000, 100000, True)